In [3]:

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
print('Library Import Complete')

Library Import Complete


In [2]:
CLIENT_ID = 'FDEU4GOTFYAIZAOVX3FTZKJOIZAZTDZTCP0X4LJE54OVU2O1' # your Foursquare ID
CLIENT_SECRET = '2OXJXJ3N5CQXKFNFWBI44GMH1JV4K222D00ZBDPXITJFEOKK' # your Foursquare Secret
VERSION = '20180604'
ACCESS_TOKEN = '2TN5DBFHOLL12RS34RFIZIYMMM5ZVJVTRHMYSJVPBQN0VOIX'
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN:' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: FDEU4GOTFYAIZAOVX3FTZKJOIZAZTDZTCP0X4LJE54OVU2O1
CLIENT_SECRET:2OXJXJ3N5CQXKFNFWBI44GMH1JV4K222D00ZBDPXITJFEOKK
ACCESS_TOKEN:2TN5DBFHOLL12RS34RFIZIYMMM5ZVJVTRHMYSJVPBQN0VOIX


## Scraping page into Dataframe

In [8]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
# create a list to store neighborhood data
neighborhoodList = []
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()
# print the number of rows of the dataframe
print("Dimension of the Dataframe: {}".format(kl_df.shape))

Dimension of the Dataframe: (71, 1)


## Getting geographical coordinates

In [9]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

coordinates

[[3.0576900000000364, 101.74388000000005],
 [3.1484921138724, 101.69672653590337],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.1357600000000616, 101.70837000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.152017197420035, 101.70102760046613],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1387586696676304, 101.6840455304707],
 [3.1387586696676304, 101.6840455304707],
 [3.156685175474611, 101.69807679

In [11]:
# merge the coordinates into the original dataframe
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coordinates['Latitude']
kl_df['Longitude'] = df_coordinates['Longitude']
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

NameError: name 'coords' is not defined